<h1 style="color:blue">Praktikum 2. Sinasõprus Pythoniga</h1>
<h3 style="color:blue">Kodeeringud, profileerimine, optimeerimine, kasulikud standardteegid</h3>
<h4>13. september 2016 </h4>
<h4>Ülesannete esitamise tähtaeg 27. september</h4>

Selles praktikumis tegeleme jätkuvalt tehniliste teemadega, kuid loomuliku keele töötluse puhul ei saa nendest üle ega ümber. 

## Kodeeringud
Keele automaatse töötluse korral tuleb alati pöörata tähelepanu kodeeringutele. Arvutil peab olema võimalus teada saada, kas kaks sõnet on identsed. Identsetete sõnede tuvastamist kasutatakse Google'i otsingumootorist mobiilis sõnade soovitamise ja teksti temaatika tuvastamiseni. Kui **käpp** ja **käpp** kasutavad erinevaid kodeeringuid, mis kodeerivad "**ä**" tähte erinevalt, on ühe kodeeringuga dekodeerides käpad arvuti jaoks erineva väärtusega ja seega erinevad.

Tuletame meelde, et mistahes tekst on arvutis bittidena. Väljundi kodeering kirjeldab, kuidas me mingit sõna (võime samastada abstraktse _unicode_'i sõnega) bittideks teisendame. Kaks erinevat kodeeringut kodeerivad (mingeid) sümboleid bittideks erinevalt. Kui me nüüd üritame erinevalt kodeeritud sõnesid ühe kodeeringu tabeli alusel dekodeerida, saame ka _unicode_'i sõned, millel me Pythonis sõne operatsioone teeme, erinevad.

Kui oleme tekstilised andmed juba vale kodeeringuga sisse lugenud, on praktikas keeruline või vähemalt väga tülikas parandusi hiljem teha. Kõrvalise informatsioonita peame halvimal juhul analüüsima dekodeerimise tulemust statistiliselt kas üle sümbolite või tekkinud sõnade (selliste lahendusteni jõuame kursuse keskel).

Andmete sisselugemisel on aga sageli kaasas lisainformatsiooni ehk andmeid andmete kohta ehk metaandmeid. Veebilehti töödeldes saame kodeeringu kätte _charset_'i päise abil, [HTTP meetodid töötavad vaikimisi üle ISO-8859-1 kodeeringu](https://www.w3.org/International/articles/http-charset/index) (oluline veebiserverit ehitades) ning tekstil võib olla ka ees [päis](https://en.wikipedia.org/wiki/Byte_order_mark), mis kirjeldab kasutatud kodeeringut.

Andmeteaduslikust vaatevinklist peame andmestiku esmalt normeerima ehk ühele kujule viima. Alles seejärel on meil lootust mingi kindlusega keerulisemaid probleeme lahendada.

Järgnevallt tutvume kodeeringutega Pythonis.


### Sõned

Python3-s on erinevat liiki sõnesid ehk sümbolite jadasid. Kõige sagedamini opereerime _unicode_'i kujul (vaikimisi).

In [3]:
unicode_string = "Pontu vaatas aknast välja\nja naeratas"
print(unicode_string)

Pontu vaatas aknast välja
ja naeratas


Mõnikord me aga ei soovi, et teatud sümboleid (nt "\") eraldi käsitletakse. Selleks on võimalik sõne defineerida _raw_ kujul.

In [5]:
raw_string = r"Pontu vaatas aknast välja\nja naeratas"
print(raw_string)

Pontu vaatas aknast välja\nja naeratas


Kodeeringute korral on oluline veel sõne kodeeritud kuju baidimassiivina, millena talletatakse kettal või liigutatakse võrgus.

In [11]:
print("Pontu vaatas aknast välja\nja naeratas".encode('utf8'))

b'Pontu vaatas aknast v\xc3\xa4lja\nja naeratas'


Kodeeritud sõnet saame dekodeerida. **_Unicode_'i ei saa dekodeerida.**

In [12]:
print(b'Pontu vaatas aknast v\xc3\xa4lja\nja naeratas'.decode('utf8'))

Pontu vaatas aknast välja
ja naeratas


### Failid

Failidega töötades oleme harjunud, et võime sõnesid lihtsalt faili kirjutada.

In [ ]:
file_name = 'my_very_special_file'

with open(file_name,'w') as fout:
    fout.write("Minu parim sõber on Juuša.")

import os
os.remove(file_name)

**Miks see töötab? Unicode'i ju ei saa kettale kirjutada :(**

Tagataustal Python dekodeerib lugedes ja kodeerib kirjutades _locale_'i eelistatud kodeeringuga. Seega, kui kasutaja ei määra ekstra kodeeringut, kasutatakse vaikekodeeringut, mida näeb järgmise käsuga:

In [1]:
# Oma vaikekodeeringu kontrollimine
import locale
locale.getpreferredencoding()

'UTF-8'

See võib teinekord hammustada. Proovime lugeda 

In [18]:
fin = open("juku.txt", "r")
fin.readlines()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x84 in position 26: invalid start byte

Kui me nurjusime vaikekodeeringuga Juku loo sisse lugemisel, on fail meie vaikekodeeringuga mitteühilduvas kodeeringus. Kui me teame, mis kodeeringus fail on, saame selle _open_ funktsioonile ette anda _encoding_ parameetriga.

## Ülesanne 1. Kodeeringud (1p)
**a) Lugeda sisse fail "juku.txt", teades, et see on salvestatud kodeeringus WINDOWS-1257. Salvestada sama fail UTF-8's. (0,5p)**

In [ ]:
with open("juku.txt", "r", encoding = "WINDOWS-1257") as fin:
    # ...

**b) Üldistada tehtut: kirjutada funktsioon, mis saab argumentidena ette failinime, algse kodeeringu, kodeeringu, millesse fail konverteerida ning uue failinime. Funktsioon salvestab etteantud faili uues kodeeringus. (0.5p)**

In [ ]:
def encoding_converter(file1, enc1, enc2, file2):
    # ...
    return "File " + file1 + " saved as " + file2 + " in encoding " + enc2

## Ülesanne 2.  Hea ja halb kood (2p)

### Mantra: 
* töötav kood >>>>>>> optimaalne kood,
* optimeerimine on vaevanõudev,
* optimeeritud kood on raskemini loetav
* optimeeri siis, kui keegi ei pea lugema või optimaalsus on hädavajalik
* pikk kood ei pruugi olla aeglane ega kompaktne kood kiire

**Kiiruse suhtes optimaalne kood:**
* suhtleb minimaalselt kettal olevate failidega,
* suhtleb veel vähem veebis olevate failidega,
* hoiab võimalikult palju relevantseid asju mälus (muutujates),
* ei sisalda tarbetuid käske, mis arvutuse kulgu ei mõjuta,
* sisaldab probleemi kontekstis mõistliku keerukusega algoritmi,
* sisaldab probleemi kontekstis õigeid andmestruktuure
     * elementide kiireks hoiustamiseks list,
     * elementide kiireks otsimiseks sõnaraamat,
     * elementide kattuvuste vms kontrollimiseks hulk,
* arvutab ühte saadust vaid ühe korra,
* sisaldab minimaalselt funktsioonide kutseid,
* sisaldab minimaalselt tingimuslausete evalueerimist,
* ei eemalda elemente listi keskelt ega viska sinna elemente

**Järgnevalt on toodud funktsioon calculate_relative_freqs, mis saab sisendiks tee failini, mille igal real on
{sõna} {absoluutne sagedus dokumendis täisarvuna}
ning mis arvutab iga sõna jaoks selle sõna suhtelise sageduse dokumendis ja tagastab tulemuse dictionary'na.**

<h5 style="color:red">NB! Siin ülesandes on näide halvast koodist, millest ei tasu mingil juhul eeskuju võtta!</h5>

**a) Vaadake koodi ja kirjeldage, mis muudab selle ebaefektiivseks (võite kirjutada vastavate koodiridade juurde kommentaaridena) (0,5p)**

In [ ]:
def calculate_relative_freqs(input_file):
    
    result = {}
    
    # Find all words for which there are frequencies
    with open(input_file, 'r') as fin:
        words = []
        for line in fin:
            words.append(line.strip().split()[1])

    # Find relative frequency for each word
    freqs = []

    for word in words:
        with open(input_file,'r') as fin:
            total_freq = 0
            word_freq = 0
            for line in fin:
                freq, word_candidate = line.strip().split()
                freqs.append(int(freq))

                if word == word_candidate:
                    word_freq += int(freq)

                total_freq += int(freq)

        freqs.sort()
        result[word] = float(word_freq) / total_freq
    return result 

**b) Kontrollige, kas teie oletustel oli tõepõhi all: selleks profileerige toodud funktsiooni ning mõelge, kas palju aega võtvad osad on vajalikud (antud kujul). Väga lihtsasti loetava tulemuse annab tööriist line_profiler, mis on küll vaja eraldi conda kaudu installida (vt loenguslaidid). Muidugi võib kasutada ka standardteeke (profile, cProfile). Millised aegluse kohta püstitatud hüpoteesidest paika pidasid? (0,5p)**

Katsetamiseks on praktikumimaterjalidesse lisatud fail nimega **lemma_kahanevas.txt**. Tegu on eesti keele sagedussõnastikuga, mis on koostatud tasakaalus korpuse pealt (vt siit http://www.cl.ut.ee/ressursid/sagedused1/).
Failis on toodud sõnade absoluutsed sagedused, et aga hinnata, kui suure osa keelest tegelikult üks või teine sõna moodustab, oleks vaja teada suhtelisi sagedusi. Seda arvutabki eeltoodud funktsioon calculate_relative_freqs.

NB! Kuna toodud koodijupp on väga ebaefektiivne, siis on soovitav profileerimiseks ja muudeks katsetusteks kasutada esialgu vaid mingit osa (nt 500 esimest rida) antud failist.

**c) Muutke kood efektiivsemaks - parandage olemasolevat või soovi korral kirjutage uus ja parem funktsioon. Kontrollige profileerides, kas sai kiirem. (1p)**

## Ülesanne 4. Kasulikud standardteegid (1p)

Siin on mõningad näited teekidest, mis muudavad loomuliku keele töötleja elu mugavamaks.

**Defaultdict** - kodeeringute loomine ja mugavad sõnaraamatud

Mugavad sõnaraamatud (näites päringumootorile tarvilik tagurpidi indeksi loomine [sõna -> dokumendi id]).
_defaultdict_ võimaldab ette anda _callable_'i, mis seab vaikeväärtuse, kui sõnaraamatus sissekanne puudub.

Sagedased kasutused: _defaultdict(int)_, _defaultdict(list)_, _defaultdict(lambda: defaultdict(int))_

In [32]:
from collections import defaultdict

sentences = ["Meri on kaunis","Kaunis koer ei tea on","On meri ja on koer"]

reverse_index = defaultdict(set)  # Default value is an empty set
for sentence_idx, sentence in enumerate(sentences):
    for word in sentence.lower().split():
        reverse_index[word].add(sentence_idx)

print(reverse_index.items())

dict_items([('tea', {1}), ('on', {0, 1, 2}), ('kaunis', {0, 1}), ('meri', {0, 2}), ('koer', {1, 2}), ('ja', {2}), ('ei', {1})])


Häkk lihtsaks teksti kodeerimiseks, et arvutustel mälu säästa.

In [25]:
from collections import defaultdict

sentence = "Habemeajaja haaras sae ja asus asja kallale ja karjus sae sae sae."
word_encoding = defaultdict()
word_encoding.default_factory = lambda: len(word_encoding)

encoded_sentence = [word_encoding[word] for word in sentence.split()]

print(word_encoding.items())
print(encoded_sentence)

dict_items([('asja', 5), ('karjus', 7), ('sae.', 8), ('Habemeajaja', 0), ('haaras', 1), ('asus', 4), ('ja', 3), ('kallale', 6), ('sae', 2)])
[0, 1, 2, 3, 4, 5, 6, 3, 7, 2, 2, 8]


**Counter** - lihtsate sagedusloendite tegemine

In [27]:
from collections import Counter

word = "habemeajaja"
letter_frequencies = Counter(word)

print(letter_frequencies.items())  # Counter has similar interface to dict
print(letter_frequencies['a'])
print(letter_frequencies['n'])  # Dict doesn't break, although 'n' doesn't exist in the initial iterable

print((letter_frequencies + Counter("ahaaaaa")).items())

dict_items([('a', 4), ('j', 2), ('h', 1), ('m', 1), ('b', 1), ('e', 2)])
4
0
dict_items([('a', 10), ('j', 2), ('h', 2), ('e', 2), ('b', 1), ('m', 1)])


**OrderedDict** - sõnastik, mis säilitab võti-väärtus-paaride järjekorra

In [30]:
from collections import OrderedDict

sorted_by_key = OrderedDict(sorted(freqs.items(), key=lambda t: t[0]))
sorted_by_value = OrderedDict(sorted(freqs.items(), key=lambda t: t[1], reverse = True))

In [31]:
sorted_by_key

OrderedDict([('a', 4), ('b', 1), ('e', 2), ('h', 1), ('j', 2), ('m', 1)])

In [32]:
sorted_by_value

OrderedDict([('a', 4), ('j', 2), ('e', 2), ('h', 1), ('b', 1), ('m', 1)])

**String.Template** ja **str.format** - lihtsad asendused sõnede sees

In [106]:
from string import Template

s = Template ("$kes tahab $mida")
s.substitute(kes = "Toomas", mida = "maja")

'Toomas tahab maja'

In [43]:
from datetime import datetime

now = datetime.now()
"Palju õnne tädi Maalile {0}. sünnipäevaks".format(now.year-1932)

'Palju õnne tädi Maalile 84. sünnipäevaks'

**str.translate** - kindlate tähemärkide asendamine

In [48]:
intab = "aeiouüõöä"
outtab = "uuuuuuuuu"
trantab = str.maketrans(intab, outtab)

str = "Põdral maja metsa sees"
str.translate(trantab)

'Pudrul muju mutsu suus'

**StringIO** - loob mällu "virtuaalse" faili, millel on _filehandler_'i liides ning kuhu on võimalik juurde kirjutada, loomata uusi objekte. Võimalik kasutada nii muudetava sõnena kui faile kasutava süsteemi testimiseks. Sõnesid saab küll kiiremini luua sõnede listi ühendades ''.join(list_of_strings).

In [33]:
from io import StringIO

virtual_file = StringIO()
virtual_file.write("Ilus sisu tuleb siit.")
virtual_file.write("\n")
virtual_file.write("Sa oled väga ilus.")
print(virtual_file.getvalue())

Ilus sisu tuleb siit.
Sa oled väga ilus.


### **re** - regulaaravaldised

<h4 style="color:green"> Regulaaravaldised on keeleandmete töötlemisel asendamatud. Kui keegi nendega tõesti veel kokku puutunud ei ole, siis on viimane aeg tutvuda https://docs.python.org/3/library/re.html ning eesti keeles https://courses.cs.ut.ee/2015/progmaa/Arhiiv/PARTVIIRegulaaravaldis1 </h4>

Regulaaravaldiste kasutamine ei piirdu Pythoni ega üleüldiselt programmeerimiskeeltega - neid on võimalik tekstis otsingute ja asenduste tegemiseks edukalt kasutada Linuxi käsurealt ('grep' käsuna), samuti mitmetes tekstitöötlusprogrammides alates MS Wordist ja lõpetades Notepad++ ja gedit'iga. 

In [56]:
import re

In [108]:
sent = "Probleemide korral saatke kiri aadressil dage@ut.ee või kom@ut.ee."
address_pattern = "[^@ .]*@[^@ .]*\.[^@ .]*"
addresses = re.findall(address_pattern, sent)
addresses

['dage@ut.ee', 'kom@ut.ee']

In [109]:
anonymised_sent = re.sub(address_pattern, "[e-mail varjatud]", sent)
anonymised_sent

'Probleemide korral saatke kiri aadressil [e-mail varjatud] või [e-mail varjatud].'

**Kirjutage skript, mis leiab failist 'protokoll.txt' tõenäolised nimed (suure tähega algavad sõnad, mis ei paikne lause alguses), teeb nendest kahanevas järjestuses sagedusloendi ning loob failist n-ö anonümiseeritud versiooni, asendades nimed millegi muuga. Failis paikneb iga lause eraldi real. Kasutage vähemalt kahte selle ülesande all tutvustatud teekidest. (1p)**

In [34]:
with open("protokoll.txt", "r", encoding="utf8") as fin:
    for line in fin:
        pass
        